# RNN in Jax

In [1]:
import jax
import jax.numpy as np
import math
import numpy as onp
import tensorflow as tf
import time
import matplotlib.pyplot as plt
from jax import grad, vmap, jit
import jax
import jax.numpy as np
import pandas as pd
import itertools
import functools
import os
from shutil import copy
from shutil import move
import pickle
from jax.config import config
config.update("jax_debug_nans", True)

In [2]:
key = jax.random.PRNGKey(1)
TRAINING_ON = False
PROCESSING_AND_DATA_LOADING_ON = False
SAVE_PARAMETERS = False
READ_PARAMETERS = True
RC_ON = False

## Pre - and Postprocessing
$$
  \mathcal{D} : \mathbf{CSV}_{\text{drum}} \rightarrow [0,1]^{K \times n_i}
$$

In [3]:
"""
Given a dataframe of a drum track and a hashmap
for mapping the drum kits to an index of the vector.
Returns the vectorized training set
consisting of the training input and output.
maxTime is for zero padding. If a signal is shorter than maxTime
then it is zero padded.
D is adjusted to also return the training output.
"""
def D(df, h, maxTime):

    time = df["time"].values
    notes = df["notes"].values
    velocity = df["velocity"].values

    dim = len(h)
    u = list(itertools.repeat(np.zeros(dim), max(time)+1))

    for i in range(len(notes)):
        v = [0]*dim
        v[h[int(notes[i])]] = int(velocity[i]) / 127
        if (np.any(u[time[i]])):        #If the current vector is not the zero vector
            u[time[i]] = u[time[i]] + np.array(v)
        else:
            u[time[i]] = np.array(v)
    
    y_train = u[1:]
    y_train.append(np.zeros(dim))

    padding = maxTime - max(time)
    return np.pad(np.array(u), ((0, padding), (0, 0)), "constant"), np.pad(np.array(y_train), ((0, padding), (0, 0)), "constant")

$$
  \mathcal{D}^{-1} : [0,1]^{K \times n_i} \rightarrow \mathbf{CSV}_{\text{drum}}
$$

In [4]:
"""
Given an output vector and a hashmap
for mapping indices of the vector to the
appropriate note/drum kit. Returns a dataframe
that can be converted to a csv file and then to a midi file
"""
def D_inv(y, h_inv):
    dfo = pd.DataFrame([], columns= ["track", "time", "control"
                            ,"channel", "notes", "velocity", "filler"])
    time_new = []
    notes_new = []
    velocity_new = []

    track_new = []
    control_new = []
    channel_new = []

    for n in range(0, len(y), 30):
        if (np.any(y[n])):
            yn = y[n]
            for i in range(len(yn)):
                v = (int)(yn[i] * 127)
                track_new.append(2)
                time_new.append(n)
                channel_new.append(9)
                notes_new.append(h_inv[i])
                velocity_new.append(v)
                if (v > 0):
                    control_new.append("Note_on_c")
                else:
                    control_new.append("Note_off_c")

    dfo["track"] = track_new
    dfo["time"] = time_new
    dfo["control"] = control_new
    dfo["channel"] = channel_new
    dfo["notes"] = notes_new
    dfo["velocity"] = velocity_new


    latestTime = time_new[-1] + 10

    pre = [[0,0, "Header", 1, 2, 480, ''],
            [1, 0, "Start_track", '', '', '', ''],
            [1, 0, "Time_signature", 4, 2, 24, 8],
            [1, 0, "Title_t", "\"from model\"", '', '', ''],
            [1, 0, "End_track", '', '', '', ''],
            [2, 0, "Start_track", '', '', '', '']]
    dfo_pre = pd.DataFrame(pre, columns =["track", "time", "control"
                            ,"channel", "notes", "velocity", "filler"])

    post = [[2, latestTime, "End_track", '', '', '', ''],
            [0, 0, "End_of_file", '', '', '', '']]

    dfo_post = pd.DataFrame(post, columns =["track", "time", "control"
                            ,"channel", "notes", "velocity", "filler"])

    # adding pre and post
    dfo = pd.concat([dfo_pre, dfo, dfo_post])

    return dfo

$$
    \mathcal{T} : \mathbf{CSV} \rightarrow \mathbf{CSV}^J_{\text{drum}}.
$$

In [5]:
"""
Returns a list of dataframes for each track
seperated by "start track" and "end track"
"""
def getTracks(df):
    subDataFrames = []
    startingIdx = -1
    # get each track as seperate dataframe
    for idx, row in df.iterrows():
        if (row["control"] == " Start_track"):
            startingIdx = idx
        elif (row["control"] == " End_track"):
            subDataFrames.append(df[startingIdx : idx + 1])
    return subDataFrames


"""
Given a dataFrame converted from a MIDI file
returns a list of dataFrames of just the channel 9 tracks
and removes all rows where the control column is
not "note_on_c"
i.e. the drum tracks
Meaning that if a song has multiple drum tracks the list
will contain more than one dataframe.
"""
def getDrumTracks(df):
    # get those tracks that are drum tracks
    drumDataFrames = [d[d["control"] == " Note_on_c"] for d in getTracks(df) if " 9" in list(d["channel"])]
    return drumDataFrames

## Data Loading

$$
    \mathcal{M} : \text{MIDI} \rightarrow \text{CSV}
$$

In [6]:

"""
Copies over the MIDI files to the folder 
where the can be converted. Once converted the CSV
files are moved to the NetworkInput CSV folder.

"""
cwd = os.getcwd()

os.chdir('../midi/NetworkInputMIDI')

fileNames = []
for m in os.listdir():
    fileNames.append(m)
    copy(m, '../midicsv-1.1')

os.chdir('../midicsv-1.1')
for m in fileNames:
    command = "midicsv" + " " + m + " " + m[:-4] + ".csv"
    res = os.system(command)
    os.remove(m)
    try:
        move(m[:-4] + ".csv", '../NetworkInputCSV')
    except:
        print("csv file already there")
        os.remove(m[:-4] + ".csv")

# ensures that the cwd resets
os.chdir(cwd)

csv file already there
csv file already there
csv file already there
csv file already there
csv file already there
csv file already there


Now import the csv files as panda dataframes (get just the drum tracks)

In [7]:
os.chdir('../midi/NetworkInputCSV')

columnNames = ["track", "time", "control"
                                , "channel", "notes", "velocity"]

"""
Loop through all the csv files in
the network input folder
"""
drumTracks_df = []
for csv in os.listdir():
    # for now: generalize later
    df = pd.read_csv(csv, skiprows=6, engine='python', names= columnNames)
    df.dropna()
    drumTracks_df = drumTracks_df + getDrumTracks(df)

os.chdir(cwd)

Create hashmap that maps notes to right vector index and additionally
1. normalize time
2. get max time

In [8]:
h = {}
h_inv = {}
unique_notes = set()

normalizingFactorFound = False
for d in drumTracks_df:
    d["notes"] = d["notes"].transform(lambda x : int(x))
    unique_notes = unique_notes.union(set(d["notes"]))
    normTime = d.iloc[0]["time"]
    d["time"] = d["time"].transform(lambda x : x - normTime)

notes = list(unique_notes)
notes.sort()

for idx in range(0, len(notes)):
    h[notes[idx]] = idx
    h_inv[idx] = notes[idx]

print(h)
print(h_inv)


{35: 0, 36: 1, 37: 2, 38: 3, 39: 4, 40: 5, 41: 6, 42: 7, 43: 8, 44: 9, 45: 10, 46: 11, 47: 12, 48: 13, 49: 14, 50: 15, 51: 16, 56: 17, 57: 18, 59: 19, 60: 20, 61: 21, 63: 22, 68: 23, 69: 24, 70: 25, 76: 26, 80: 27, 81: 28, 82: 29, 87: 30, 88: 31, 89: 32, 91: 33, 93: 34}
{0: 35, 1: 36, 2: 37, 3: 38, 4: 39, 5: 40, 6: 41, 7: 42, 8: 43, 9: 44, 10: 45, 11: 46, 12: 47, 13: 48, 14: 49, 15: 50, 16: 51, 17: 56, 18: 57, 19: 59, 20: 60, 21: 61, 22: 63, 23: 68, 24: 69, 25: 70, 26: 76, 27: 80, 28: 81, 29: 82, 30: 87, 31: 88, 32: 89, 33: 91, 34: 93}


Seperate drum tracks into bars every 2222.222 ms is a bar
$$
    \mathcal{B} : \mathbf{CSV}^J_{\text{drum}} \rightarrow \mathbf{CSV}^B_{\text{drum}}
$$

In [9]:
def seperateIntoBars(df, barLength=2222):
    subDataFrames = []
    startingTime = 0
    startingIdx = 0
    idx = 0
    for _, row in df.iterrows():
        if (row["time"] - startingTime > barLength and (row["time"] - startingTime < barLength + 1000)) :
            subDataFrames.append(df[startingIdx : idx + 1].copy())
            startingTime = row["time"]
            startingIdx = idx
        idx = idx + 1
    
    for d in subDataFrames:
        normTime = d.iloc[0]["time"]
        d["time"] = d["time"].transform(lambda x : x - normTime)
    return subDataFrames

df_bars = []
for df in drumTracks_df:
    df_bars = df_bars + seperateIntoBars(df)

maxTime = -1
for d in df_bars:
    for idx, row in d.iterrows():
        if (row["time"] > maxTime):
            maxTime = row["time"]

print(len(drumTracks_df))
print(len(df_bars))

20
744


Get vectors from drum track dataframes

In [10]:
if (PROCESSING_AND_DATA_LOADING_ON):
    S = []  # S will be a list of tuples
            # each tuple contains two input signals over time

    for df in df_bars:
        u_train, y_train = D(df, h, maxTime)
        S.append((u_train, y_train))

    u_ex, y_ex = S[0]
    print(jax.tree_map(lambda x: x.shape, u_ex))
    print(jax.tree_map(lambda x: x.shape, y_ex))

Save vectors so we do not have to rerun it

In [11]:
# https://pynative.com/python-write-list-to-file/
# write list to binary file
def write_list(a_list, name):
    # store list in binary file so 'wb' mode
    with open(name, 'wb') as fp:
        pickle.dump(a_list, fp)
        print('Done writing list into a binary file')

# Read list to memory
def read_list(name):
    # for reading also binary mode is important
    with open(name, 'rb') as fp:
        n_list = pickle.load(fp)
        return n_list


In [12]:
if (PROCESSING_AND_DATA_LOADING_ON):
    write_list(S, "training")
else:
    S = read_list("training")

# Network Architecture

A RNN (with no LSTM units) is a neural network with recurrent connections (dynamical system)
$$
\begin{array}{l l}
          \mathbf{x}(n+1) = \sigma(W \mathbf{x}(n) + W^{in}\mathbf{u}(n+1) + \mathbf{b}) \\
          \mathbf{y}(n) = f(W^{out} \mathbf{x}(n)),
\end{array}
$$
Describes how the network activation state is updated and how output signal is generated. 

Input vector $\mathbf{u}(n) \in [0,1]^K$

Activation/state vector $\mathbf{x}(n) \in \mathbb{R}^L$

Output vector $\mathbf{y}(n) \in \mathbb{R}^K$

Bias vector $\mathbf{b} \in \mathbb{R}^L$

$W^{in} \in \mathbb{R}^{L \times K}, W \in \mathbb{R}^{L \times L}, W^{out} \in \mathbb{R}^{K \times L}$ are weight matrices charecterizing the connections between neurons in the layers


## Initialization
At time $n = 0$ the recurrent network state $\mathbf{x}(0)$ is often set to the zero vector $\mathbf{x}(0) = \mathbf{0}$

In [13]:
def init_weight_matrix(in_dim, out_dim, key, scale=1e-2):
    w = jax.random.normal(key, (out_dim, in_dim))
    return scale*w

In [14]:
def init_bias(dim, key, scale=1e-2):
    b = jax.random.normal(key, (dim, ))
    return scale*b

In [15]:
# state, params = (W^{in}, W, W^{out}, b)
# sizes = (input dim, state dim, output dim.)
def init_network(sizes, key):
    keys = jax.random.split(key, len(sizes))
    
    Win = init_weight_matrix(sizes[0], sizes[1], keys[0])
    W = init_weight_matrix(sizes[1], sizes[1], keys[1])
    Wout = init_weight_matrix(sizes[1], sizes[2], keys[3])
    b = init_bias(sizes[1], keys[2])

    return (Win, W, Wout, b)


In [16]:
K = len(S[0][0][0]) # K := input and output vector dim
L = 1000 # Reservoir or State Vector dim
sizes = [K, L, K]
params = init_network(sizes, key)
print(jax.tree_map(lambda x: x.shape, params)) # printing shape of network

((1000, 35), (1000, 1000), (35, 1000), (1000,))


## Training


In [17]:

# derived from https://github.com/rug-minds/teaching/blob/main/boilerplate/reservoir.ipynb
# x_og is initially the zero vector
def forward_bp(params, u, x_og=np.zeros((L, ))):
    """ Loop over the time steps of the input sequence
    u[n] := [u_0, ..., u_{n_max}] where u_i \in [0, 1]^K or (K, )
    x_og: \in R^L or (L, )
    """
    Win, W, Wout, b = params
    x = x_og.copy()

    def apply_fun_scan(params, x, ut):
        """ Perform single step update of the network.
        x:  (L, )
        un: (K, )
        """
        Win, W, Wout, b = params
        x = jax.nn.sigmoid(
            np.dot(Win, ut) + np.dot(W, x) + b
        )
        y = jax.nn.sigmoid(np.dot(Wout, x))
        return x, y     # this returns nan at some point according to nan debugger

    f = functools.partial(apply_fun_scan, params)
    _, Y = jax.lax.scan(f, x, u)
    return Y

batch_forward_bp = jax.vmap(forward_bp, in_axes=(None, 0))

## Loss function
Logistic regression is used for the case where you do not model loudness. Because there it is a categorical task (to hit or not to hit). But if you do model loudness you use the procedure linear regression. Meaning using the loss functions that are mentioned in the RNN section of the reader like MSE or quadratic loss.

Time series prediction task $S = (\mathbf{u}^{(i)}(n), \mathbf{y}^{(i)}(n))_{i=1, ..., N;n=1, ..., n_i}$ where $\mathbf{y}^{(i)}(n) = \mathbf{u}^{(i)}(n+1)$.
Quadratic loss which is used in stationary tasks
$$
    L(\hat{\mathbf{Y}}_{i, \theta}^{\text{train}}, \mathbf{Y}_i^{\text{train}}) = \parallel \hat{\mathbf{Y}}_{i, \theta}^{\text{train}} - \mathbf{Y}_i^{\text{train}} \parallel^2
$$

Regularization
$$
\text{reg}(\theta) = \sum_{w \in \theta} w^2
$$

In [18]:
def reg(params):
    Win, W, Wout, b = params
    return (
        np.sum(np.square(Win)) +
        np.sum(np.square(W)) +
        np.sum(np.square(Wout)) +
        np.sum(np.square(b))
    )


$$
\mathcal{R}^{\text{emp}}(\theta) = \frac{1}{N} \sum^{N}_{i=1} L(\hat{\mathbf{Y}}_{i, \theta}^{\text{train}}, \mathbf{Y}_i^{\text{train}}) + r^2 \; \text{reg}(\theta)
$$

In [19]:
def loss(params, u, y_true, alpha):
    y_hat = batch_forward_bp(params, u)
    return np.square(np.linalg.norm(np.subtract(y_hat, y_true))) + (alpha*alpha)*reg(params)

$$
    \theta^{(n+1)} = \theta^{(n)} - \mu \nabla \mathcal{R}^{\text{emp}}(\theta^{(n)}),
$$

$$
   \nabla \mathcal{R}^{\text{emp}}(\theta^{(n)}) = 
\bigg(\frac{\partial  R^{emp}}{\partial  w_1}(\theta^{(n)}), ...,\frac{\partial  R^{emp}}{\partial w_D}(\theta^{(n)}) \bigg)',
$$

In [20]:
"""
gradient descent update
"""
@jax.jit
def update(params, u, y_true, alpha, step_size=1e-2):
    grads = jax.grad(loss)(params, u, y_true, alpha)

    return [
        (w - step_size * dw)
        for w, dw in zip(params, grads)
    ]

$$
    \mathcal{A}(S) = \theta_{\text{opt}} = \underset{\theta \in \Theta}{\text{argmin}} \; \underbrace{\frac{1}{N} \sum^N_{i=1} L(\hat{\mathbf{Y}}_{\theta}^{\text{train}}, \mathbf{Y}^{\text{train}})}_{\mathcal{R}^{\text{emp}}(\theta)},
$$

## Cross Validation

In [21]:

"""
s is a list of tuples 
[(u1, y1), (u2, y2), ...] -> ((u_1, u_2, ...), (y_1, y_2, ...))
a signal u_i or y_i is (K, n_i) where n_i is the length of the song which can differ
and K is the number of modeled drum parts
"""
def unpack(s):
    f1 = map(lambda x: x[0], s)
    f2 = map(lambda x: x[1], s)

    u_batch = np.array(list(f1))
    y_batch = np.array(list(f2))

    return (u_batch, y_batch)

"""
trains the network on the training data for n epochs
in the cross validation setup the testing data is
the validation set and is simply used to measure the
testing loss next to the training loss
"""
def train(params, u_train, y_train, u_test, y_test, alpha, n_epochs=15, batch_size=128):
    train_loss = []
    test_loss = []
    for epoch in range(n_epochs):
        start_time = time.time()
        n_batches = math.ceil(u_train.shape[0] / batch_size)
        for batch_idx in range(n_batches):
            start = batch_idx * batch_size
            end = -1 if batch_idx == (n_batches - 1) else start + batch_size
            u_batch = u_train[start:end, :]
            y_batch = y_train[start:end, :]
            params = update(params, u_batch, y_batch, alpha)

        train_loss.append(loss(params, u_train, y_train, alpha))
        test_loss.append(loss(params, u_test, y_test, alpha))

        epoch_time = time.time() - start_time

        print(f'Epoch {epoch+1:>2} ({epoch_time:<.2f}s): ', end='')
        print(f'train loss {train_loss[-1]:<5.2f} test loss {test_loss[-1]:<5.2f}', end='| ')
    
    return params, train_loss, test_loss

"""
same as train but without testing set to measure testing loss
"""
def train2(params, u_train, y_train, r, n_epochs=2, batch_size=128):
    train_loss = []
    for epoch in range(n_epochs):
        start_time = time.time()
        n_batches = math.ceil(u_train.shape[0] / batch_size)
        for batch_idx in range(n_batches):
            start = batch_idx * batch_size
            end = -1 if batch_idx == (n_batches - 1) else start + batch_size
            u_batch = u_train[start:end, :]
            y_batch = y_train[start:end, :]
            params = update(params, u_batch, y_batch, r)

        train_loss.append(loss(params, u_train, y_train, r))

        epoch_time = time.time() - start_time

        print(f'Epoch {epoch+1:>2} ({epoch_time:<.2f}s): ', end='')
        print(f'train loss {train_loss[-1]:<5.2f} |')
    return params

if (TRAINING_ON and not RC_ON):
    # given S, params, loss/emprical risk, r (regularization)
    # Hyperparameters
    print(len(S))
    step_size=1e-2
    k = 10   # k fold cross validation
    # r is the hyperparameter (here r = alpha for the regularization)
    validation_risk_r = []

    """
    Edge case: if k > amount of tuples (u_train, y_train) in S then n = 0
    which will result in a valueError when constructing S_k.
    Furthermore: the procedure will break down for single (u_train, y_train)
    because then the reduced training set will be the empty set
    """
    n = (int) (len(S)/k)

    # split S into k disjoint subsets
    S_k = [S[i : i + n] for i in range(0, len(S), n)]


    full_train_loss = []
    full_test_loss = [] # loss on validation set
    for r in range(0, 5):
        validation_risk = []
        for j in range(0, k):
            V = S_k.pop(j)                          # validation set
            T = [x for l in S_k for x in l]         # reduced training set
            S_k.insert(j, V)
            u_train, y_train = unpack(T)
            #print(u_train.shape)
            #print(y_train.shape)
            u_val, y_val = unpack(V)
            #print(u_val.shape)
            #print(y_val.shape)
            params, train_loss, test_loss = train(params, u_train, y_train, u_val, y_val, r)
            # record loss / emp.risk
            full_train_loss = full_train_loss + train_loss
            full_test_loss = full_test_loss + test_loss

            validation_risk.append(loss(params, u_val, y_val, r))

        validation_risk_r.append(np.mean(np.array(validation_risk)))

    r_opt = np.argmin(np.array(validation_risk_r))
    u_train, y_train = unpack([x for l in S_k for x in l])  # train on whole training set
    params = train2(params, u_train, y_train, r_opt)

### Parameter Saving

In [22]:
if (READ_PARAMETERS):
    params = tuple(read_list("params"))
elif (SAVE_PARAMETERS):
    write_list("params")

## Training results plotting

In [23]:
if (TRAINING_ON):  
    fig, axs = plt.subplots()
    fig.suptitle("Performance of RNN on one-time step prediction of drum tracks (using JAX)")
    axs.set_title('Loss')
    axs.plot(full_train_loss, label='train loss')
    axs.plot(full_test_loss, label='test loss')
    axs.set_ylabel('Quadratic loss')
    axs.set_xlabel('Training epochs')
    axs.set_ylim(0, 1.6e9)
    axs.legend()

    plt.plot()

In [24]:
if (TRAINING_ON):
    fig, axs = plt.subplots()
    axs.set_title('validation risk')
    axs.plot(validation_risk_r, label='validation risk')
    axs.set_ylabel('validation risk')
    axs.set_xlabel('alpha parameter')
    axs.legend()

    plt.plot()

In [33]:
print(len(S[0][0]))

3181


## Music Generation
Get the file to prime the network with

In [34]:
def apply_fun_scan(params, x, un):
    """ Perform single step update of the network.
    x:  (L, ) at time step n -> x: (L, ) at time step n+1
    un: (K, )
    """
    Win, W, Wout, b = params
    x = np.tanh(
        np.dot(Win, un) + np.dot(W, x) + b
    )
    y = jax.nn.sigmoid(np.dot(Wout, x))
    return x, y

y_signal = []

#song = D(drumTracks_df[0], h,  151960)[0]
selection = [S[0][0], S[1][0], S[2][0]]
for u_prime in selection:
    n_stop = len(u_prime)
    
    x = np.zeros((L,))
    for n in range(n_stop):
        x, y = apply_fun_scan(params, x, u_prime[n])
        y_signal.append(y)

n_output = 20000
for n in range(n_output):
    x, y = apply_fun_scan(params, x, y)
    y_signal.append(y)

dfo = D_inv(np.array(y_signal), h_inv)

$$
\mathcal{M}^{-1} : \text{CSV} \rightarrow \text{MIDI}
$$

In [35]:
"""
Puts generated CSV file in correct folder
and generates the MIDI file and puts that in the correct
folder as well.
"""

os.chdir('../midi/NetworkOutputCSV')
name = "rnn.csv"
dfo.to_csv(name, index = False, header = False)
copy(name, '../midicsv-1.1')
os.chdir('../midicsv-1.1')

command = "csvmidi" + " " + name + " " + name[:-4] + ".mid"
res = os.system(command)
print(res)
os.remove(name)
try:
    move(name[:-4] + ".mid", '../NetworkOutputMIDI')
except:
    print("midi file already here")
    os.remove(name[:-4] + ".mid")

os.chdir(cwd)

0


## Experimental: reservoir computing
dimensionality $K, L, K$ same: using same rnn.
First: scaling with spectral radius

In [52]:

# derived from https://github.com/rug-minds/teaching/blob/main/boilerplate/reservoir.ipynb
if (RC_ON):
    W, Win, Wout, b = params
    rhoW = np.max(np.absolute(np.linalg.eig(W)[0]))
    scale = 1.25
    W = (scale/rhoW)*W

Step 1: state harvesting: record activation for each of the $L$ reservoir neurons
when driven by teacher input

In [57]:
# derived from https://github.com/rug-minds/teaching/blob/main/boilerplate/reservoir.ipynb
def forward(u, Win, W, b, Wout=None, x_init=np.zeros((L, ))):
    # u: (n_max, K)
    n_max = u.shape[0]
    K = u.shape[1]
    X, Y = [], []
    # X: (1+K+L, n_max)
    # Y: (K, n_max) double chheck later
    x = x_init.copy()
    for t in range(u.shape[0]):
        x = np.tanh(
            np.dot(Win, u[t]) + np.dot(W, x) + b
        )
        full_state = np.concatenate(u[t], x, b)
        X.append(full_state)
        if Wout is not None:
            y = np.dot(Wout, full_state)
            Y.append(y)
        # when returning: need to transpose data arrays, such that dim: (n_max, x)
    if Wout is None:
        return x, np.array(X).T
    else:
        return x, np.array(X).T, np.array(Y).T

In [58]:
# derived from https://github.com/rug-minds/teaching/blob/main/boilerplate/reservoir.ipynb
if (RC_ON):
    x, X = forward(u_train[0], Win, W)
    print(f'u: {u_train[0].shape}, x: {x.shape}, X: {X.shape}')

step 2: compute readouts

In [56]:
# derived from https://github.com/rug-minds/teaching/blob/main/boilerplate/reservoir.ipynb
if (RC_ON):
    reg = 1e-8

    Wout_rc_ = np.dot(
        np.dot(y_train[0].T, X.T),
        np.linalg.inv(
            np.dot(X, X.T) + reg*np.eye(1+K+L)
        )
    )